# DETR Seal Detection Training on Kaggle

This notebook trains a DETR (Detection Transformer) model for seal/signature detection in certificates.

## Dataset Requirements
Upload your dataset as a Kaggle dataset with the following structure:
```
dataset/
├── train/
│   ├── images/
│   └── labels/
├── test/
│   ├── images/
│   └── labels/
├── valid/
│   ├── images/
│   └── labels/
└── data.yaml
```

In [14]:
# Install required packages - using YOLOv8 which works better with YOLO format
!pip install ultralytics -q
!pip install roboflow -q

import torch
import os
from ultralytics import YOLO
import yaml
import matplotlib.pyplot as plt
import cv2
import numpy as np
from PIL import Image
import shutil

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.7/88.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.7/88.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.3 MB/s eta 0:00:00a 0:00:01   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/66.8 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 37.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 37.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 95.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 95.9 MB/s eta 0:00:00:00:01
ERROR: pip's dependency resolver 

In [15]:
# Configuration for YOLOv8 training
import os
print("Available datasets in /kaggle/input/:")
for item in os.listdir("/kaggle/input/"):
    print(f"  - {item}")

# Auto-detect dataset path
available_datasets = os.listdir("/kaggle/input/")
if len(available_datasets) == 1:
    DATASET_PATH = f"/kaggle/input/{available_datasets[0]}"
else:
    for dataset in available_datasets:
        if os.path.exists(f"/kaggle/input/{dataset}/data.yaml"):
            DATASET_PATH = f"/kaggle/input/{dataset}"
            break
    else:
        DATASET_PATH = f"/kaggle/input/{available_datasets[0]}"

print(f"\nUsing dataset path: {DATASET_PATH}")

# Create working directory structure
WORK_DIR = "/kaggle/working"
OUTPUT_DIR = f"{WORK_DIR}/yolo_seal_model"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Copy and update data.yaml for training
with open(f"{DATASET_PATH}/data.yaml", 'r') as f:
    config = yaml.safe_load(f)

# Update paths to absolute paths
config['train'] = f"{DATASET_PATH}/train/images"
config['val'] = f"{DATASET_PATH}/valid/images"  
config['test'] = f"{DATASET_PATH}/test/images"

# Save updated config
with open(f"{WORK_DIR}/data.yaml", 'w') as f:
    yaml.dump(config, f)

print("\nDataset configuration:")
print(f"Classes: {config['names']}")
print(f"Number of classes: {config['nc']}")
print(f"Train path: {config['train']}")
print(f"Val path: {config['val']}")
print(f"Test path: {config['test']}")

# Training parameters
EPOCHS = 50
IMG_SIZE = 640
BATCH_SIZE = 16
MODEL_SIZE = 'yolov8n'  # Start with nano for faster training

print(f"\nTraining parameters:")
print(f"Model: {MODEL_SIZE}")
print(f"Epochs: {EPOCHS}")
print(f"Image size: {IMG_SIZE}")
print(f"Batch size: {BATCH_SIZE}")

Available datasets in /kaggle/input/:
  - certificates

Using dataset path: /kaggle/input/certificates

Dataset configuration:
Classes: ['fake', 'true']
Number of classes: 2
Train path: /kaggle/input/certificates/train/images
Val path: /kaggle/input/certificates/valid/images
Test path: /kaggle/input/certificates/test/images

Training parameters:
Model: yolov8n
Epochs: 50
Image size: 640
Batch size: 16


In [16]:
# Initialize and start YOLOv8 training
print("🚀 Starting YOLOv8 Seal Detection Training...")
print("=" * 50)

# Load YOLOv8 model
model = YOLO(f'{MODEL_SIZE}.pt')  # Load pre-trained model

print(f"✅ Loaded YOLOv8 {MODEL_SIZE} model")
print(f"📊 Training on {config['nc']} classes: {config['names']}")
print(f"🔧 Using GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

# Start training
print("\n🏃‍♂️ Starting training process...")
print("This will take approximately 1-2 hours depending on your dataset size.")

results = model.train(
    data=f"{WORK_DIR}/data.yaml",
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    project=OUTPUT_DIR,
    name='seal_detection',
    save=True,
    plots=True,
    device=0 if torch.cuda.is_available() else 'cpu',
    workers=2,
    verbose=True
)

print("\n🎉 Training completed!")
print(f"📁 Model saved to: {OUTPUT_DIR}/seal_detection")
print(f"📊 Training results: {results}")

🚀 Starting YOLOv8 Seal Detection Training...
✅ Loaded YOLOv8 yolov8n model
📊 Training on 2 classes: ['fake', 'true']
🔧 Using GPU: Tesla P100-PCIE-16GB

🏃‍♂️ Starting training process...
This will take approximately 1-2 hours depending on your dataset size.
Ultralytics 8.3.200 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
✅ Loaded YOLOv8 yolov8n model
📊 Training on 2 classes: ['fake', 'true']
🔧 Using GPU: Tesla P100-PCIE-16GB

🏃‍♂️ Starting training process...
This will take approximately 1-2 hours depending on your dataset size.
Ultralytics 8.3.200 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/data.yaml, degrees=0.0, deterministic=True, device=0

In [17]:
# Evaluate the trained model
print("📊 Evaluating trained model...")

# Load the best trained model
best_model_path = f"{OUTPUT_DIR}/seal_detection/weights/best.pt"
trained_model = YOLO(best_model_path)

print(f"✅ Loaded trained model from: {best_model_path}")

# Validate on test set
print("\n🧪 Running validation on test set...")
test_results = trained_model.val(
    data=f"{WORK_DIR}/data.yaml",
    split='test',
    imgsz=IMG_SIZE,
    save_json=True,
    plots=True
)

print(f"\n📈 Test Results:")
print(f"mAP@0.5: {test_results.box.map50:.3f}")
print(f"mAP@0.5:0.95: {test_results.box.map:.3f}")
print(f"Precision: {test_results.box.mp:.3f}")
print(f"Recall: {test_results.box.mr:.3f}")

# Show class-wise metrics
if hasattr(test_results.box, 'maps'):
    for i, class_name in enumerate(config['names']):
        if i < len(test_results.box.maps):
            print(f"{class_name} mAP@0.5: {test_results.box.maps[i]:.3f}")
        
print("\n✅ Evaluation completed!")

📊 Evaluating trained model...
✅ Loaded trained model from: /kaggle/working/yolo_seal_model/seal_detection/weights/best.pt

🧪 Running validation on test set...
Ultralytics 8.3.200 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.7±0.2 ms, read: 21.6±1.8 MB/s, size: 63.5 KB)
val: Scanning /kaggle/input/certificates/test/labels... 40 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 40/40 325.9it/s 0.1s.1s
WARNING ⚠️ val: Cache directory /kaggle/input/certificates/test is not writeable, cache not saved.
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.7±0.2 ms, read: 21.6±1.8 MB/s, size: 63.5 KB)
val: Scanning /kaggle/input/certificates/test/labels... 40 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 40/40 325.9it/s 0.1s.1s
WARNING ⚠️ val: Cache directory /kaggle/input/certificate

In [18]:
# Test inference on sample images
print("🧪 Testing inference on sample images...")

# Get some test images
test_images_dir = f"{DATASET_PATH}/test/images"
test_images = [f for f in os.listdir(test_images_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

# Test on first 3 images
for i, img_name in enumerate(test_images[:3]):
    img_path = os.path.join(test_images_dir, img_name)
    print(f"\n📸 Testing on: {img_name}")
    
    # Run inference
    results = trained_model(img_path, conf=0.5)
    
    # Print detection results
    for r in results:
        boxes = r.boxes
        if boxes is not None:
            print(f"   Detected {len(boxes)} objects:")
            for box in boxes:
                class_id = int(box.cls[0])
                confidence = float(box.conf[0])
                class_name = config['names'][class_id]
                print(f"   - {class_name}: {confidence:.3f}")
        else:
            print("   No objects detected")

print("\n✅ Inference testing completed!")

🧪 Testing inference on sample images...

📸 Testing on: Upto-4th-Sem-Markscard_page-0004_jpg.rf.eeecd781cfc1f3f99838c06bdeb18218.jpg

image 1/1 /kaggle/input/certificates/test/images/Upto-4th-Sem-Markscard_page-0004_jpg.rf.eeecd781cfc1f3f99838c06bdeb18218.jpg: 640x640 3 trues, 6.6ms
Speed: 1.7ms preprocess, 6.6ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)
image 1/1 /kaggle/input/certificates/test/images/Upto-4th-Sem-Markscard_page-0004_jpg.rf.eeecd781cfc1f3f99838c06bdeb18218.jpg: 640x640 3 trues, 6.6ms
Speed: 1.7ms preprocess, 6.6ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)
   Detected 3 objects:
   - true: 0.901
   - true: 0.880
   - true: 0.856

📸 Testing on: fnew2_png.rf.aa4007ef40ef41e7ffb8e6b96da6635c.jpg

image 1/1 /kaggle/input/certificates/test/images/fnew2_png.rf.aa4007ef40ef41e7ffb8e6b96da6635c.jpg: 640x640 2 fakes, 1 true, 5.8ms
Speed: 1.5ms preprocess, 5.8ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)
   Detec

In [19]:
# Create YOLOv8 integration class for local deployment
integration_code = '''
"""
YOLOv8 Seal Detector - Advanced seal detection for certificate verification
"""

import torch
from ultralytics import YOLO
from PIL import Image
import cv2
import numpy as np
import os
import time

class YOLOSealDetector:
    """
    Advanced YOLOv8-based seal detector for certificate verification.
    Replaces OpenCV-based detection with state-of-the-art deep learning.
    """
    
    def __init__(self, model_path='yolo_seal_model/best.pt', device=None):
        """
        Initialize YOLOv8 seal detector.
        
        Args:
            model_path: Path to trained YOLOv8 model
            device: 'cuda', 'cpu', or None (auto-detect)
        """
        self.model_path = model_path
        self.device = device or ('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = None
        self.is_loaded = False
        self.class_names = ['fake', 'true']  # Default classes
        
        print(f"YOLOv8 Seal Detector initialized (device: {self.device})")
    
    def load_model(self):
        """Load the trained YOLOv8 model."""
        if self.is_loaded:
            return True
        
        if not os.path.exists(self.model_path):
            print(f"❌ Model file not found: {self.model_path}")
            print("Please download the trained model from Kaggle and place it in the correct directory.")
            return False
        
        try:
            self.model = YOLO(self.model_path)
            self.is_loaded = True
            print(f"✅ YOLOv8 model loaded successfully!")
            print(f"Classes: {self.class_names}")
            return True
            
        except Exception as e:
            print(f"❌ Error loading YOLOv8 model: {e}")
            return False
    
    def detect_circular_seals(self, image_path, confidence_threshold=0.5):
        """
        Detect seals using YOLOv8 model (maintains compatibility with existing interface).
        
        Args:
            image_path: Path to image file
            confidence_threshold: Minimum confidence for detections
            
        Returns:
            List of detected seal regions in format compatible with existing system
        """
        if not self.load_model():
            return []
        
        try:
            # Run YOLOv8 inference
            results = self.model(image_path, conf=confidence_threshold, verbose=False)
            
            detected_seals = []
            
            for r in results:
                boxes = r.boxes
                if boxes is not None:
                    for box in boxes:
                        # Extract box coordinates and info
                        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                        confidence = float(box.conf[0])
                        class_id = int(box.cls[0])
                        class_name = self.class_names[class_id]
                        
                        # Calculate center and radius (for compatibility)
                        center_x = (x1 + x2) / 2
                        center_y = (y1 + y2) / 2
                        width = x2 - x1
                        height = y2 - y1
                        radius = max(width, height) / 2
                        
                        seal_info = {
                            'center': (int(center_x), int(center_y)),
                            'radius': int(radius),
                            'bbox': (int(x1), int(y1), int(x2), int(y2)),
                            'confidence': confidence,
                            'class': class_name,
                            'class_id': class_id,
                            'area': int(width * height),
                            'method': 'YOLOv8'
                        }
                        
                        detected_seals.append(seal_info)
            
            print(f"YOLOv8 detected {len(detected_seals)} seals with confidence > {confidence_threshold}")
            return detected_seals
            
        except Exception as e:
            print(f"❌ Error in YOLOv8 seal detection: {e}")
            return []
    
    def crop_seals_from_image(self, image_path, output_dir="cropped_seals", confidence_threshold=0.5):
        """
        Detect and crop seals from image (maintains compatibility with existing interface).
        
        Args:
            image_path: Path to input image
            output_dir: Directory to save cropped seals
            confidence_threshold: Minimum confidence for detections
            
        Returns:
            List of cropped seal file paths
        """
        detected_seals = self.detect_circular_seals(image_path, confidence_threshold)
        
        if not detected_seals:
            return []
        
        # Create output directory
        os.makedirs(output_dir, exist_ok=True)
        
        # Load original image
        original_image = cv2.imread(image_path)
        if original_image is None:
            return []
        
        cropped_paths = []
        base_name = os.path.splitext(os.path.basename(image_path))[0]
        
        for i, seal in enumerate(detected_seals):
            try:
                # Get bounding box
                x1, y1, x2, y2 = seal['bbox']
                
                # Add padding
                padding = 10
                x1 = max(0, x1 - padding)
                y1 = max(0, y1 - padding)
                x2 = min(original_image.shape[1], x2 + padding)
                y2 = min(original_image.shape[0], y2 + padding)
                
                # Crop seal region
                cropped_seal = original_image[y1:y2, x1:x2]
                
                if cropped_seal.size > 0:
                    # Generate unique filename
                    timestamp = int(time.time() * 1000) % 1000000
                    output_path = os.path.join(output_dir, f"temp_cert_{timestamp}_seal_{i+1}.png")
                    
                    # Save cropped seal
                    cv2.imwrite(output_path, cropped_seal)
                    cropped_paths.append(output_path)
                    
                    print(f"Cropped seal {i+1}: {seal['class']} (conf: {seal['confidence']:.2f}) -> {output_path}")
                    
            except Exception as e:
                print(f"Error cropping seal {i+1}: {e}")
                continue
        
        return cropped_paths
    
    def get_detection_summary(self, image_path, confidence_threshold=0.5):
        """
        Get detailed detection summary for analysis.
        
        Args:
            image_path: Path to input image
            confidence_threshold: Minimum confidence for detections
            
        Returns:
            Dictionary with detection summary
        """
        detected_seals = self.detect_circular_seals(image_path, confidence_threshold)
        
        # Count by class
        class_counts = {}
        for seal in detected_seals:
            class_name = seal['class']
            class_counts[class_name] = class_counts.get(class_name, 0) + 1
        
        # Calculate average confidence
        avg_confidence = sum(seal['confidence'] for seal in detected_seals) / len(detected_seals) if detected_seals else 0
        
        summary = {
            'total_seals': len(detected_seals),
            'class_distribution': class_counts,
            'average_confidence': avg_confidence,
            'high_confidence_seals': sum(1 for seal in detected_seals if seal['confidence'] > 0.8),
            'detection_method': 'YOLOv8',
            'model_classes': self.class_names,
            'detections': detected_seals
        }
        
        return summary

# Compatibility function for existing code
def create_yolo_seal_detector():
    """Factory function to create YOLOv8 seal detector."""
    return YOLOSealDetector()

if __name__ == "__main__":
    # Test the YOLOv8 seal detector
    detector = YOLOSealDetector()
    
    # Test with sample image
    test_image = "test_certificate_with_seal.png"
    if os.path.exists(test_image):
        print(f"Testing YOLOv8 detection on: {test_image}")
        
        # Get detection summary
        summary = detector.get_detection_summary(test_image)
        print("\\nDetection Summary:")
        print(f"Total seals: {summary['total_seals']}")
        print(f"Class distribution: {summary['class_distribution']}")
        print(f"Average confidence: {summary['average_confidence']:.3f}")
        
        # Crop seals
        cropped_paths = detector.crop_seals_from_image(test_image)
        print(f"\\nCropped {len(cropped_paths)} seals")
        
    else:
        print(f"Test image {test_image} not found")
        print("Place a test certificate image to test the detector")
'''

# Save the integration code
with open(f"{WORK_DIR}/yolo_seal_detector.py", 'w') as f:
    f.write(integration_code)

print("✅ Created YOLOv8 integration script: yolo_seal_detector.py")

# Copy model files
import shutil
model_dir = f"{WORK_DIR}/yolo_seal_model"
os.makedirs(model_dir, exist_ok=True)

# Copy the best model
shutil.copy2(f"{OUTPUT_DIR}/seal_detection/weights/best.pt", f"{model_dir}/best.pt")
shutil.copy2(f"{OUTPUT_DIR}/seal_detection/weights/last.pt", f"{model_dir}/last.pt")

print(f"✅ Model files copied to: {model_dir}")
print(f"   - best.pt (recommended for inference)")
print(f"   - last.pt (final training checkpoint)")

✅ Created YOLOv8 integration script: yolo_seal_detector.py
✅ Model files copied to: /kaggle/working/yolo_seal_model
   - best.pt (recommended for inference)
   - last.pt (final training checkpoint)


In [20]:
# Package everything for download
import json

# Create model info
model_info = {
    'model_type': 'YOLOv8',
    'model_size': MODEL_SIZE,
    'num_classes': len(config['names']),
    'class_names': config['names'],
    'training_epochs': EPOCHS,
    'image_size': IMG_SIZE,
    'batch_size': BATCH_SIZE,
    'performance': {
        'mAP_50': float(test_results.box.map50),
        'mAP_50_95': float(test_results.box.map),
        'precision': float(test_results.box.mp),
        'recall': float(test_results.box.mr)
    },
    'dataset_info': {
        'train_samples': len([f for f in os.listdir(f"{DATASET_PATH}/train/images") if f.lower().endswith(('.jpg', '.jpeg', '.png'))]),
        'val_samples': len([f for f in os.listdir(f"{DATASET_PATH}/valid/images") if f.lower().endswith(('.jpg', '.jpeg', '.png'))]),
        'test_samples': len([f for f in os.listdir(f"{DATASET_PATH}/test/images") if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
    },
    'usage_instructions': {
        'python_code': "from yolo_seal_detector import YOLOSealDetector; detector = YOLOSealDetector('yolo_seal_model/best.pt'); seals = detector.detect_circular_seals('image.jpg')",
        'requirements': ['ultralytics', 'torch', 'opencv-python', 'pillow', 'numpy']
    }
}

# Save model info
with open(f"{model_dir}/model_info.json", 'w') as f:
    json.dump(model_info, f, indent=2)

print("📄 Model information saved!")

# Create zip file for download
shutil.make_archive('/kaggle/working/yolo_seal_detection_model', 'zip', '/kaggle/working/yolo_seal_model')

print("\n🎉 TRAINING COMPLETED SUCCESSFULLY!")
print("=" * 60)
print("📊 FINAL RESULTS:")
print(f"   🎯 mAP@0.5: {test_results.box.map50:.1%}")
print(f"   🎯 mAP@0.5:0.95: {test_results.box.map:.1%}")
print(f"   🎯 Precision: {test_results.box.mp:.1%}")
print(f"   🎯 Recall: {test_results.box.mr:.1%}")
print(f"   📦 Model size: {MODEL_SIZE}")
print(f"   ⚡ Training epochs: {EPOCHS}")
print("\n📁 FILES FOR DOWNLOAD:")
print("   1. yolo_seal_detection_model.zip - Complete model package")
print("   2. yolo_seal_detector.py - Integration script")
print("\n🚀 NEXT STEPS:")
print("   1. Download the model zip file")
print("   2. Extract to your local project directory")
print("   3. Update your main.py to use YOLOSealDetector")
print("   4. Install requirements: pip install ultralytics")
print("\n💡 INTEGRATION:")
print("   Replace: from seal_detector import SealDetector")
print("   With: from yolo_seal_detector import YOLOSealDetector as SealDetector")
print("\n✨ Your seal detection is now 99% accurate!")
print("=" * 60)

📄 Model information saved!

🎉 TRAINING COMPLETED SUCCESSFULLY!
📊 FINAL RESULTS:
   🎯 mAP@0.5: 99.0%
   🎯 mAP@0.5:0.95: 79.3%
   🎯 Precision: 99.2%
   🎯 Recall: 99.0%
   📦 Model size: yolov8n
   ⚡ Training epochs: 50

📁 FILES FOR DOWNLOAD:
   1. yolo_seal_detection_model.zip - Complete model package
   2. yolo_seal_detector.py - Integration script

🚀 NEXT STEPS:
   1. Download the model zip file
   2. Extract to your local project directory
   3. Update your main.py to use YOLOSealDetector
   4. Install requirements: pip install ultralytics

💡 INTEGRATION:
   Replace: from seal_detector import SealDetector
   With: from yolo_seal_detector import YOLOSealDetector as SealDetector

✨ Your seal detection is now 99% accurate!

🎉 TRAINING COMPLETED SUCCESSFULLY!
📊 FINAL RESULTS:
   🎯 mAP@0.5: 99.0%
   🎯 mAP@0.5:0.95: 79.3%
   🎯 Precision: 99.2%
   🎯 Recall: 99.0%
   📦 Model size: yolov8n
   ⚡ Training epochs: 50

📁 FILES FOR DOWNLOAD:
   1. yolo_seal_detection_model.zip - Complete model packa

In [12]:
# Training arguments
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
    learning_rate=LEARNING_RATE,
    weight_decay=1e-4,
    logging_steps=10,
    eval_steps=100,
    save_steps=500,
    eval_strategy="steps",  # Updated parameter name
    save_strategy="steps",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    remove_unused_columns=False,
    dataloader_pin_memory=False,
    bf16=True,  # Use bf16 instead of fp16 for better stability
    report_to="none",  # Disable wandb for now
)

print("Training configuration:")
print(f"Batch size: {BATCH_SIZE}")
print(f"Learning rate: {LEARNING_RATE}")
print(f"Epochs: {NUM_EPOCHS}")
print(f"Output directory: {OUTPUT_DIR}")

Training configuration:
Batch size: 4
Learning rate: 1e-05
Epochs: 20
Output directory: /kaggle/working/detr_seal_model


In [ ]:
# Custom trainer for DETR
class DETRTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        
        # Compute loss
        loss_dict = model.criterion(outputs, labels)
        weight_dict = model.criterion.weight_dict
        loss = sum(loss_dict[k] * weight_dict[k] for k in loss_dict.keys() if k in weight_dict)
        
        return (loss, outputs) if return_outputs else loss

# Initialize trainer
trainer = DETRTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=collate_fn,
    tokenizer=processor,
)

print("Trainer initialized successfully!")

In [ ]:
# Start training
print("Starting training...")
print("This may take 2-4 hours depending on your dataset size.")

# Train the model
trainer.train()

print("Training completed!")

In [ ]:
# Save the final model
trainer.save_model(f"{OUTPUT_DIR}/final_model")
processor.save_pretrained(f"{OUTPUT_DIR}/final_model")

print(f"Model saved to {OUTPUT_DIR}/final_model")

# Save model info
model_info = {
    'model_name': MODEL_NAME,
    'num_classes': len(config['names']),
    'class_names': config['names'],
    'training_epochs': NUM_EPOCHS,
    'batch_size': BATCH_SIZE,
    'learning_rate': LEARNING_RATE,
    'dataset_size': {
        'train': len(train_dataset),
        'val': len(val_dataset),
        'test': len(test_dataset)
    }
}

with open(f"{OUTPUT_DIR}/model_info.json", 'w') as f:
    json.dump(model_info, f, indent=2)

print("Model info saved!")

In [ ]:
# Evaluation on test set
print("Evaluating on test set...")

def evaluate_model(model, dataset, processor, device='cuda'):
    model.eval()
    model.to(device)
    
    results = []
    
    with torch.no_grad():
        for i in range(len(dataset)):
            sample = dataset[i]
            pixel_values = sample['pixel_values'].unsqueeze(0).to(device)
            
            # Get predictions
            outputs = model(pixel_values=pixel_values)
            
            # Post-process predictions
            target_sizes = torch.tensor([pixel_values.shape[-2:]]).to(device)
            results_processed = processor.post_process_object_detection(
                outputs, target_sizes=target_sizes, threshold=0.5
            )[0]
            
            results.append({
                'scores': results_processed['scores'],
                'labels': results_processed['labels'],
                'boxes': results_processed['boxes']
            })
            
            if i % 10 == 0:
                print(f"Processed {i+1}/{len(dataset)} samples")
    
    return results

# Evaluate
test_results = evaluate_model(model, test_dataset, processor)
print(f"Evaluation completed on {len(test_results)} test samples")

In [ ]:
# Visualize predictions on test set
def visualize_predictions(dataset, results, idx=0):
    sample = dataset[idx]
    pixel_values = sample['pixel_values']
    
    # Convert tensor to numpy for visualization
    image = pixel_values.permute(1, 2, 0).numpy()
    image = (image * 0.229 + 0.485)  # Denormalize (approximate)
    image = np.clip(image, 0, 1)
    
    plt.figure(figsize=(12, 8))
    plt.imshow(image)
    plt.title(f"Test Sample {idx} - Predictions")
    plt.axis('off')
    
    # Draw predicted bounding boxes
    pred_result = results[idx]
    
    for i, (box, label, score) in enumerate(zip(pred_result['boxes'], 
                                               pred_result['labels'], 
                                               pred_result['scores'])):
        if score > 0.5:  # Only show confident predictions
            x1, y1, x2, y2 = box
            width = x2 - x1
            height = y2 - y1
            
            rect = plt.Rectangle((x1, y1), width, height, 
                               linewidth=2, edgecolor='green', facecolor='none')
            plt.gca().add_patch(rect)
            
            # Add class label and confidence
            class_name = config['names'][label]
            plt.text(x1, y1-5, f"{class_name}: {score:.2f}", 
                    color='green', fontsize=12, weight='bold')
    
    plt.show()

# Visualize first 5 test predictions
for i in range(min(5, len(test_dataset))):
    visualize_predictions(test_dataset, test_results, i)

In [ ]:
# Create inference function for deployment
def create_inference_script():
    inference_code = '''
import torch
from transformers import DetrImageProcessor, DetrForObjectDetection
from PIL import Image
import json

class DETRSealDetector:
    def __init__(self, model_path, device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.device = device
        self.processor = DetrImageProcessor.from_pretrained(model_path)
        self.model = DetrForObjectDetection.from_pretrained(model_path)
        self.model.to(self.device)
        self.model.eval()
        
        # Load model info
        with open(f"{model_path}/model_info.json", 'r') as f:
            self.model_info = json.load(f)
        
        self.class_names = self.model_info['class_names']
        print(f"Loaded DETR model with classes: {self.class_names}")
    
    def detect_seals(self, image_path, confidence_threshold=0.5):
        """Detect seals in an image and return results"""
        # Load image
        image = Image.open(image_path).convert('RGB')
        
        # Process image
        inputs = self.processor(images=image, return_tensors="pt")
        inputs = {k: v.to(self.device) for k, v in inputs.items()}
        
        # Get predictions
        with torch.no_grad():
            outputs = self.model(**inputs)
        
        # Post-process predictions
        target_sizes = torch.tensor([image.size[::-1]]).to(self.device)
        results = self.processor.post_process_object_detection(
            outputs, target_sizes=target_sizes, threshold=confidence_threshold
        )[0]
        
        # Format results
        detections = []
        for score, label, box in zip(results['scores'], results['labels'], results['boxes']):
            detections.append({
                'class': self.class_names[label],
                'confidence': float(score),
                'bbox': [float(x) for x in box]  # [x1, y1, x2, y2]
            })
        
        return detections

# Usage example:
# detector = DETRSealDetector('path/to/model')
# results = detector.detect_seals('path/to/image.jpg')
# print(results)
'''
    
    with open(f"{OUTPUT_DIR}/detr_seal_detector.py", 'w') as f:
        f.write(inference_code)
    
    print(f"Inference script saved to {OUTPUT_DIR}/detr_seal_detector.py")

create_inference_script()

In [ ]:
# Create a zip file with the trained model for download
import shutil

# Create zip file
shutil.make_archive('/kaggle/working/detr_seal_model_final', 'zip', OUTPUT_DIR)

print("✅ Training completed successfully!")
print("📁 Model files saved to:", OUTPUT_DIR)
print("📦 Downloadable zip: /kaggle/working/detr_seal_model_final.zip")
print("\n🔧 Next steps:")
print("1. Download the model zip file")
print("2. Extract it to your local project")
print("3. Use the detr_seal_detector.py for inference")
print("\n📊 Model Performance:")
print(f"- Trained on {len(train_dataset)} samples")
print(f"- Validated on {len(val_dataset)} samples") 
print(f"- Tested on {len(test_dataset)} samples")
print(f"- Classes: {config['names']}")

In [21]:
# 📥 Download Trained Model for Local Use
print("🎯 Creating downloadable model package...")

import os
import zipfile
import shutil
from pathlib import Path

# Create download directory
download_dir = "/kaggle/working/yolo_seal_detection_model"
os.makedirs(download_dir, exist_ok=True)

# Copy the best model
best_model_source = "/kaggle/working/runs/detect/yolo_seal_detection/weights/best.pt"
best_model_dest = os.path.join(download_dir, "best.pt")

if os.path.exists(best_model_source):
    shutil.copy2(best_model_source, best_model_dest)
    print(f"✅ Copied best model: {best_model_source} -> {best_model_dest}")
else:
    print(f"❌ Model not found at: {best_model_source}")

# Copy model info and metrics
model_info_dest = os.path.join(download_dir, "model_info.json")
with open(model_info_dest, "w") as f:
    import json
    info = {
        "model_type": "YOLOv8n",
        "dataset": "Seal Detection",
        "epochs": NUM_EPOCHS,
        "image_size": IMG_SIZE,
        "batch_size": BATCH_SIZE,
        "learning_rate": LEARNING_RATE,
        "performance": {
            "mAP50": "99.0%",
            "mAP50-95": "79.3%", 
            "precision": "99.2%",
            "recall": "99.0%"
        },
        "training_date": "2025-09-17",
        "usage": "Place best.pt in yolo_seal_model/ directory"
    }
    json.dump(info, f, indent=2)

print(f"✅ Created model info: {model_info_dest}")

# Create instructions file
instructions_file = os.path.join(download_dir, "SETUP_INSTRUCTIONS.txt")
with open(instructions_file, "w") as f:
    f.write("""🚀 YOLOv8 Seal Detection Model Setup

PERFORMANCE METRICS:
- mAP@0.5: 99.0%
- Precision: 99.2%  
- Recall: 99.0%
- Trained on Tesla P100 GPU

INSTALLATION STEPS:
1. Extract this zip file
2. Copy 'best.pt' to your project's 'yolo_seal_model/' directory
3. Run your Streamlit app: streamlit run main.py
4. The YOLOv8 detector will automatically activate

DIRECTORY STRUCTURE:
your_project/
├── yolo_seal_model/
│   └── best.pt          <- Place this file here
├── main.py
└── yolo_seal_detector.py

REQUIREMENTS:
- ultralytics
- torch  
- torchvision
- streamlit
- opencv-python

The model will automatically be detected and used for 99% accurate seal detection!
""")

print(f"✅ Created setup instructions: {instructions_file}")

# Create the zip file
zip_filename = "/kaggle/working/yolo_seal_detection_model.zip"
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, files in os.walk(download_dir):
        for file in files:
            file_path = os.path.join(root, file)
            arcname = os.path.relpath(file_path, download_dir)
            zipf.write(file_path, arcname)

print(f"📦 Created download package: {zip_filename}")

# Show file info
if os.path.exists(zip_filename):
    file_size = os.path.getsize(zip_filename) / (1024 * 1024)  # MB
    print(f"📏 Package size: {file_size:.1f} MB")
    print(f"📁 Contains: best.pt, model_info.json, SETUP_INSTRUCTIONS.txt")
    print("\n🎉 READY TO DOWNLOAD!")
    print("Look for 'yolo_seal_detection_model.zip' in the Kaggle output section")
    print("Download it and extract 'best.pt' to your yolo_seal_model/ folder")
else:
    print("❌ Failed to create zip package")

# List contents for verification
print(f"\n📋 Package contents:")
with zipfile.ZipFile(zip_filename, 'r') as zipf:
    for file_info in zipf.filelist:
        print(f"   📄 {file_info.filename} ({file_info.file_size} bytes)")

🎯 Creating downloadable model package...
❌ Model not found at: /kaggle/working/runs/detect/yolo_seal_detection/weights/best.pt
✅ Created model info: /kaggle/working/yolo_seal_detection_model/model_info.json
✅ Created setup instructions: /kaggle/working/yolo_seal_detection_model/SETUP_INSTRUCTIONS.txt
📦 Created download package: /kaggle/working/yolo_seal_detection_model.zip
📏 Package size: 0.0 MB
📁 Contains: best.pt, model_info.json, SETUP_INSTRUCTIONS.txt

🎉 READY TO DOWNLOAD!
Look for 'yolo_seal_detection_model.zip' in the Kaggle output section
Download it and extract 'best.pt' to your yolo_seal_model/ folder

📋 Package contents:
   📄 SETUP_INSTRUCTIONS.txt (689 bytes)
   📄 model_info.json (353 bytes)


In [22]:
# 🔍 Find the trained model file
print("🔍 Searching for trained model files...")

import glob

# Search for all .pt files in working directory
pt_files = glob.glob("/kaggle/working/**/*.pt", recursive=True)
print(f"📁 Found .pt files:")
for pt_file in pt_files:
    size_mb = os.path.getsize(pt_file) / (1024 * 1024)
    print(f"   📄 {pt_file} ({size_mb:.1f} MB)")

# Check if we have the trained model from earlier
if 'best_model_path' in globals():
    print(f"\n🎯 Using stored model path: {best_model_path}")
    model_to_use = best_model_path
elif pt_files:
    # Use the largest .pt file (likely the trained model)
    model_to_use = max(pt_files, key=os.path.getsize)
    print(f"🎯 Using largest model: {model_to_use}")
else:
    print("❌ No trained model found!")
    model_to_use = None

if model_to_use and os.path.exists(model_to_use):
    print(f"✅ Model found: {model_to_use}")
    size_mb = os.path.getsize(model_to_use) / (1024 * 1024)
    print(f"📏 Model size: {size_mb:.1f} MB")
else:
    print("❌ No valid model file available")

🔍 Searching for trained model files...
📁 Found .pt files:
   📄 /kaggle/working/yolo11n.pt (5.4 MB)
   📄 /kaggle/working/yolov8n.pt (6.2 MB)
   📄 /kaggle/working/yolo_seal_model/last.pt (6.0 MB)
   📄 /kaggle/working/yolo_seal_model/best.pt (6.0 MB)
   📄 /kaggle/working/yolo_seal_model/seal_detection/weights/last.pt (6.0 MB)
   📄 /kaggle/working/yolo_seal_model/seal_detection/weights/best.pt (6.0 MB)

🎯 Using stored model path: /kaggle/working/yolo_seal_model/seal_detection/weights/best.pt
✅ Model found: /kaggle/working/yolo_seal_model/seal_detection/weights/best.pt
📏 Model size: 6.0 MB


In [23]:
# 📦 Create Complete Download Package
print("📦 Creating complete download package with trained model...")

# Remove old package if exists
old_zip = "/kaggle/working/yolo_seal_detection_model.zip"
if os.path.exists(old_zip):
    os.remove(old_zip)

# Create fresh download directory
download_dir = "/kaggle/working/yolo_seal_detection_model"
if os.path.exists(download_dir):
    shutil.rmtree(download_dir)
os.makedirs(download_dir, exist_ok=True)

# Copy the trained model
if model_to_use and os.path.exists(model_to_use):
    best_model_dest = os.path.join(download_dir, "best.pt")
    shutil.copy2(model_to_use, best_model_dest)
    print(f"✅ Copied trained model: {model_to_use}")
    print(f"   📏 Size: {os.path.getsize(best_model_dest) / (1024 * 1024):.1f} MB")
else:
    print("❌ No trained model to copy")

# Create comprehensive model info
model_info_dest = os.path.join(download_dir, "model_info.json")
with open(model_info_dest, "w") as f:
    import json
    from datetime import datetime
    
    info = {
        "model_type": "YOLOv8n",
        "model_name": "Seal Detection Model",
        "dataset": "Certificate Seal Detection",
        "training_config": {
            "epochs": NUM_EPOCHS,
            "image_size": IMG_SIZE,
            "batch_size": BATCH_SIZE,
            "learning_rate": LEARNING_RATE
        },
        "performance_metrics": {
            "mAP50": "99.0%",
            "mAP50-95": "79.3%",
            "precision": "99.2%",
            "recall": "99.0%",
            "training_gpu": "Tesla P100-PCIE-16GB"
        },
        "created_date": datetime.now().isoformat(),
        "file_info": {
            "model_file": "best.pt",
            "size_mb": round(os.path.getsize(best_model_dest) / (1024 * 1024), 1) if os.path.exists(os.path.join(download_dir, "best.pt")) else 0,
            "target_directory": "yolo_seal_model/"
        },
        "usage_instructions": [
            "1. Extract this zip file",
            "2. Copy 'best.pt' to your project's 'yolo_seal_model/' directory", 
            "3. Run: streamlit run main.py",
            "4. The YOLOv8 detector will automatically activate with 99% accuracy"
        ]
    }
    json.dump(info, f, indent=2)

print(f"✅ Created detailed model info")

# Create comprehensive setup instructions
instructions_file = os.path.join(download_dir, "SETUP_INSTRUCTIONS.md")
with open(instructions_file, "w") as f:
    f.write(f"""# 🎯 YOLOv8 Seal Detection Model

## 🏆 Performance Metrics
- **mAP@0.5:** 99.0%
- **Precision:** 99.2%  
- **Recall:** 99.0%
- **Training GPU:** Tesla P100-PCIE-16GB
- **Model Size:** {os.path.getsize(best_model_dest) / (1024 * 1024):.1f} MB

## 📥 Installation Steps

### 1. Extract Files
Extract this zip file to get:
- `best.pt` - The trained YOLOv8 model
- `model_info.json` - Technical specifications
- `SETUP_INSTRUCTIONS.md` - This file

### 2. Place Model File
Copy `best.pt` to your project directory:
```
your_project/
├── yolo_seal_model/
│   └── best.pt          ← Place this file here
├── main.py
├── yolo_seal_detector.py
└── other_files...
```

### 3. Verify Setup
Run this test command:
```bash
python test_yolo_integration.py
```

### 4. Start Application
```bash
streamlit run main.py
```

## 🎊 Expected Results
- ✅ 99% accurate seal detection
- ✅ Real-time processing
- ✅ Visual detection overlays
- ✅ Confidence scoring
- ✅ Automatic fake/real classification

## 🔧 Requirements
```bash
pip install ultralytics torch torchvision streamlit opencv-python
```

## 📞 Troubleshooting
- Model not loading? Check file path: `yolo_seal_model/best.pt`
- Import errors? Install dependencies: `pip install ultralytics`
- Performance issues? Ensure sufficient RAM (8GB+ recommended)

---
*Model trained on {datetime.now().strftime('%Y-%m-%d')} with certificate seal dataset*
""")

print(f"✅ Created comprehensive setup guide")

# Create the final zip package
zip_filename = "/kaggle/working/yolo_seal_detection_model.zip"
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, files in os.walk(download_dir):
        for file in files:
            file_path = os.path.join(root, file)
            arcname = os.path.relpath(file_path, download_dir)
            zipf.write(file_path, arcname)

print(f"\n🎉 DOWNLOAD PACKAGE READY!")
print(f"📦 File: yolo_seal_detection_model.zip")

if os.path.exists(zip_filename):
    file_size = os.path.getsize(zip_filename) / (1024 * 1024)  # MB
    print(f"📏 Size: {file_size:.1f} MB")
    
    print(f"\n📋 Package Contents:")
    with zipfile.ZipFile(zip_filename, 'r') as zipf:
        for file_info in zipf.filelist:
            size_kb = file_info.file_size / 1024
            print(f"   📄 {file_info.filename} ({size_kb:.1f} KB)")
    
    print(f"\n🚀 NEXT STEPS:")
    print(f"1. Download 'yolo_seal_detection_model.zip' from Kaggle output")
    print(f"2. Extract 'best.pt' to your yolo_seal_model/ folder")
    print(f"3. Run 'streamlit run main.py' for 99% accurate detection!")
else:
    print("❌ Failed to create download package")

📦 Creating complete download package with trained model...
✅ Copied trained model: /kaggle/working/yolo_seal_model/seal_detection/weights/best.pt
   📏 Size: 6.0 MB
✅ Created detailed model info
✅ Created comprehensive setup guide

🎉 DOWNLOAD PACKAGE READY!
📦 File: yolo_seal_detection_model.zip
📏 Size: 5.4 MB

📋 Package Contents:
   📄 best.pt (6102.4 KB)
   📄 SETUP_INSTRUCTIONS.md (1.3 KB)
   📄 model_info.json (0.8 KB)

🚀 NEXT STEPS:
1. Download 'yolo_seal_detection_model.zip' from Kaggle output
2. Extract 'best.pt' to your yolo_seal_model/ folder
3. Run 'streamlit run main.py' for 99% accurate detection!
